In [37]:
# Set up helper functions
from sensor_msgs.msg import JointState
from std_msgs.msg import Float64MultiArray
import rosbag
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

# master loader for the bag, returns the messages on topic_name in time order
def bag_loader(path, map_and_append_msg, topic_name = 'robot_state', normalize = ''):
    # map_and_append_msgs is a function handle which should be written for each type.  There are some for joint state and robot state below
    bag = rosbag.Bag(path)
    num_obs = bag.get_message_count(topic_name)
    if num_obs is 0:
        topic_name = '/'+topic_name
        num_obs = bag.get_message_count(topic_name)
    print('Loading ros bag {}  with {} msgs on topic {}'.format(path, num_obs, topic_name))

    msgs = {}
    t = []
    for _, msg, t_ros in bag.read_messages(topics=[topic_name]):
        t.append(t_ros.to_sec())
        map_and_append_msg(msg, msgs)
    t = [tt-t[0] for tt in t]

    msgs_in_order = {}
    for key in msgs.keys():
        t_in_order, el_in_order = zip(*sorted(zip(t,msgs[key])))
        msgs_in_order[key] = np.array(el_in_order).T
    msgs_in_order['t'] = t_in_order
    
    if normalize is not '':
        msgs_in_order[normalize] = (msgs_in_order[normalize].T-msgs_in_order[normlize][:,0]).T
        
    return msgs_in_order

def get_aligned_msgs(msgs1, msgs2):
    ''' 
    Select entries from msgs2 which occured most recently before msgs1
    '''
    aligned_msgs2 = {key:[] for key in msgs2.keys()}
    t2 = np.array(msgs2['t'])
    for t1 in msgs1['t']:
        last_before_t1 = np.where(t2<=t1)[0][-1] # find last time in t which is 
        for key in msgs2.keys():
            if key == 't': continue
            aligned_msgs2[key].append(msgs2[key][:,last_before_t1])

    for key in msgs2.keys():
        aligned_msgs2[key] = np.array(aligned_msgs2[key]).T
    
    return aligned_msgs2

def map_robot_state(msg, prev_msgs):
    if len(prev_msgs) is 0:
        for el in ('pos', 'vel', 'force'):
            prev_msgs[el] = []
    prev_msgs['pos'].append(msg.position)
    prev_msgs['vel'].append(msg.velocity)
    prev_msgs['force'].append(msg.effort)
    return prev_msgs


def map_joint_state(msg, prev_msgs):
    if len(prev_msgs) is 0:
        for el in ('pos', 'name', 'time'):
            prev_msgs[el] = []
    prev_msgs['pos'].append(msg.position)
    prev_msgs['name'].append(msg.name)
    return prev_msgs

def map_wrench(msg, prev_msgs):
    #print(f"Debug: {len(prev_msgs)}")
    if len(prev_msgs) is 0:
        prev_msgs['force'] = []
    prev_msgs['force'].append(np.array([msg.wrench.force.x, 
                                        msg.wrench.force.y,
                                        msg.wrench.force.z]))
    return prev_msgs

def map_point_msgs(msg, prev_msgs):
    #print(f"Debug: {len(prev_msgs)}")
    if len(prev_msgs) is 0:
        prev_msgs['position'] = []
    #print(f"Debug: {msg.pose.__dir__()}")
    prev_msgs['position'].append(np.array([msg.pose.position.x, 
                                        msg.pose.position.y,
                                        msg.pose.position.z]))
    return prev_msgs
    

def map_impedance_gains(msg, prev_msgs):
    if len(prev_msgs) is 0:
        for el in ('K', 'B', 'M', 'Fd'):
            prev_msgs[el] = []
    prev_msgs['K'].append(msg.position)
    prev_msgs['B'].append(msg.velocity)
    prev_msgs['M'].append(msg.effort[:len(msg.position)])
    prev_msgs['Fd'].append(msg.effort[len(msg.position):])

def map_delta_impedance_gains(msg, prev_msgs):
    if len(prev_msgs) is 0:
        for el in ('dK', 'dB', 'dM', 'Fd'):
            prev_msgs[el] = []
    prev_msgs['dK'].append(msg.position)
    prev_msgs['dB'].append(msg.velocity)
    prev_msgs['dM'].append(msg.effort[:len(msg.position)])
    prev_msgs['Fd'].append(msg.effort[len(msg.position):])
    

<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:32: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:55: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:75: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:84: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:94: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:103: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:32: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:55: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:75: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:84: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:94: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:103: SyntaxWarning: "is" with a literal. Did you mean "=="

In [38]:
import yaml
from rosbag.bag import Bag
bag_path = '/home/christian/Projects/contact_monitoring/data/plug_threading_online.bag'
bag_dict = yaml.load(Bag(bag_path, 'r')._get_yaml_info(), Loader=yaml.Loader)
print("Keys are")
for k in bag_dict['topics']:
    #print(f"\n{k}:\n{bag_dict[k]}")
    print(f"\n:\n{k}")


Keys are

:
{'topic': '/cartesian_compliance_controller/parameter_descriptions', 'type': 'dynamic_reconfigure/ConfigDescription', 'messages': 1}

:
{'topic': '/cartesian_compliance_controller/parameter_updates', 'type': 'dynamic_reconfigure/Config', 'messages': 1}

:
{'topic': '/cartesian_compliance_controller/pd_gains/rot_x/parameter_descriptions', 'type': 'dynamic_reconfigure/ConfigDescription', 'messages': 1}

:
{'topic': '/cartesian_compliance_controller/pd_gains/rot_x/parameter_updates', 'type': 'dynamic_reconfigure/Config', 'messages': 1}

:
{'topic': '/cartesian_compliance_controller/pd_gains/rot_y/parameter_descriptions', 'type': 'dynamic_reconfigure/ConfigDescription', 'messages': 1}

:
{'topic': '/cartesian_compliance_controller/pd_gains/rot_y/parameter_updates', 'type': 'dynamic_reconfigure/Config', 'messages': 1}

:
{'topic': '/cartesian_compliance_controller/pd_gains/rot_z/parameter_descriptions', 'type': 'dynamic_reconfigure/ConfigDescription', 'messages': 1}

:
{'topic':

In [39]:
### Comparing with and without variance in contact
# %matplotlib

#topics
#'contact_mode': JointState message with name, position where positoin is the similarity
#'wrench' # WrenchStamped FT measurements
#'target_frame', # desired pose for admittance ctrl
#'target_wrench', # desired force for admittance trl
         

path = '/home/christian/Projects/contact_monitoring/data/'
bag = 'plug_threading_online.bag'

contact_state_msgs = bag_loader(path+bag, map_joint_state, topic_name = 'contact_mode')
FT_msgs = bag_loader(path+bag, map_wrench, topic_name = 'wrench')
des_FT_msgs = bag_loader(path+bag, map_wrench, topic_name = 'target_wrench')
des_pose_msgs = bag_loader(path+bag, map_point_msgs, topic_name = 'target_frame')

Loading ros bag /home/christian/Projects/contact_monitoring/data/plug_threading_online.bag  with 41233 msgs on topic /contact_mode


AttributeError: '_sensor_msgs__JointState' object has no attribute 't'

In [62]:
FT = FT_msgs['force']
Contact = contact_state_msgs['name'][0,:]

Contact[10] = "not_free_space"
#for name in Contact[0,:]:
#    if name != "free_space":
#            print(name)


Contact_types = []
for name in Contact:
    if not any(name in types for types in Contact_types):
        Contact_types.append(name)
print(Contact_types)
        
        
contact_state_msgs_frequency = 488.4197
T_msg_contact = 1/contact_state_msgs_frequency
time_contact = np.arange(0,contact_state_msgs['pos'].shape[1]*T_msg_contact,T_msg_contact)

FT_msgs_frequency = 454.4208
T_msg_FT = 1/FT_msgs_frequency
time_FT = np.arange(0, FT_msgs['force'].shape[1]*T_msg_FT, T_msg_FT)




['free_space', 'not_free_spac']


In [ ]:
fig, (ax0, ax1) = plt.subplots(2, 1)

ax0.plot(time_FT, FT)
ax1.psd(s, 512, 1 / dt)

plt.show()